In [2]:
import pandas as pd
import numpy as np

import pylab as pl
from matplotlib import pyplot as plt
import seaborn as sns

%pylab inline

!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
from tqdm import tqdm_notebook as tqdm

Populating the interactive namespace from numpy and matplotlib
     - 501kB 4.3MB/s
  Stored in directory: /tmp/pip-ephem-wheel-cache-3thkbtaa/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
fastai 1.0.50.post1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [3]:
import time
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/dev-set/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [4]:
!ls '/content/drive/My Drive/dev-set/'

aesthetic_visual_features  dev-set_ground-truth.csv    HOG	    ORB
C3D			   dev-set_video-captions.txt  InceptionV3  README.txt
ColorHistogram		   HMP			       LBP


In [5]:
ground_truth_file = '/content/drive/My Drive/dev-set/dev-set_ground-truth.csv'
ground_truth = pd.read_csv(ground_truth_file)
ground_truth.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10


In [6]:
!pip install pyprind

In [0]:
# load labels and captions
def read_caps(fname):
    """Load the captions into a dataframe"""
    vn = []
    cap = []
    df = pd.DataFrame();
    with open(fname) as f:
        for line in f:
            pairs = line.split()
            vn.append(pairs[0])
            cap.append(pairs[1])
        df['video']=vn
        df['caption']=cap
    return df


# load the captions
from pathlib import Path
inception_path = Path('InceptionV3')

# load the ground truth values
label_path = './'
labels=pd.read_csv(label_path+'dev-set_ground-truth.csv')

In [17]:
from pathlib import Path
inception_path = Path('InceptionV3')

!ls '{inception_path}' | wc -l


18110


In [18]:
example_inception_path = inception_path / 'video10-0.txt'
example_inception_path.open().read()


'8:2.811053e-38 12:4.527463e-34 58:7.063336e-34 107:4.006941e-31 110:9.993380e-01 112:4.186179e-35 123:1.304208e-37 294:7.722262e-37 295:2.903859e-32 323:6.014483e-33 392:5.602231e-38 396:4.263801e-33 524:1.684702e-38 543:1.156429e-24 546:3.005845e-19 611:4.177478e-38 618:3.339212e-32 693:2.644775e-29 714:6.619816e-04 725:3.289480e-21 765:2.632254e-36 773:5.967941e-14 811:2.472793e-36 850:5.449528e-35 852:1.594492e-28 857:4.170681e-37 864:3.997572e-36 883:1.215084e-17 899:2.787453e-31 951:1.757438e-27 961:9.840043e-27 984:7.116916e-26 \n'

In [0]:
def parse_inception_feature(s):
    pairs = s.strip().split(' ')
    pairs = [i.split(':') for i in pairs]
    return {int(k): float(v) for k, v in pairs}

sample_inception_dictionary = parse_inception_feature(example_inception_path.open().read())
sample_inception_dictionary


In [0]:
def expand_inception_feature(d):
  feature = np.zeros(1000)
  for k, v in d.items():
    feature[k] = v
  return feature
  
# show first 20 values
expand_inception_feature(sample_inception_dictionary)[:20]


In [21]:

sum(expand_inception_feature(sample_inception_dictionary))

0.9999999816000597

In [22]:
example_inception_path.with_suffix('.webm').name.replace('-0', '')

'video10.webm'

In [0]:
max_iterations = 20


In [0]:
list(inception_path.glob('*-0.txt'))[:max_iterations]

In [0]:
max_iterations = 6000

inception_features_dict = {}
for file in tqdm(list(inception_path.glob('*-0.txt'))[:max_iterations]):
  # only process first frame of each video
  key = file.with_suffix('.webm').name.replace('-0', '')
  inception_features_dict[key] = expand_inception_feature(
    parse_inception_feature(
      file.open().read()))

In [0]:
# convert features to dataframe
inception_features = pd.DataFrame(inception_features_dict).T
inception_features.head(10)


In [0]:
inception_features.loc['video1297.webm'].values[:20]


In [0]:
print("features: ", inception_features)
 

In [30]:
inception_features.shape

(6000, 1000)

In [58]:
print("Labels: ", inception_features.values)

Labels:  [[0.000000e+00 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 6.646033e-30 0.000000e+00
  0.000000e+00]
 ...
 [5.632873e-33 3.536131e-23 0.000000e+00 ... 7.218544e-29 0.000000e+00
  1.282517e-36]
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 1.694470e-31 0.000000e+00
  0.000000e+00]]


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import preprocessing
from sklearn import utils
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

In [94]:
inception_features.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
video10.webm,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,2.811053e-38,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0
video100.webm,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0
video1001.webm,0.0,0.000000e+00,0.0,0.0,0.0,6.735891e-33,0.0,0.0,3.607844e-25,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,6.646033e-30,0.0,0.0
video1003.webm,0.0,1.035399e-33,0.0,0.0,0.0,4.483960e-35,0.0,0.0,2.748289e-34,0.0,...,0.0,0.0,0.0,3.737847e-38,0.0,0.0,0.0,0.000000e+00,0.0,0.0
video1004.webm,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,2.384511e-31,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,1.164084e-37,0.0,0.0


In [95]:
inception_features.columns


RangeIndex(start=0, stop=1000, step=1)

In [0]:
feature_cols = (inception_features.columns)

In [0]:
X = inception_features[feature_cols] # Features
y = inception_features.values # Target variable

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [0]:
from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor(criterion = 'mse')

In [144]:
clf.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [0]:
y_pred =  clf.predict(X_test)

In [153]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
print('Accuracy Score on train data: ', accuracy_score(y_true=y_train, y_pred=clf.predict(X_train)))
print('Accuracy Score on test data: ', accuracy_score(y_true=y_test, y_pred=clf.predict(X_train)))


ValueError: ignored